In [3]:
import jieba
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [4]:
with open('天龙八部.txt', 'r',encoding='utf-8') as f:
    text = f.read()
print(text[:1000])

一 青衫磊落险峰行

　　青光闪动，一柄青钢剑倏地刺出，指向中年汉子左肩，使剑少年不待剑招用老，腕抖剑斜，剑锋已削向那汉子右颈。那中年汉子竖剑挡格，铮的一声响，双剑相击，嗡嗡做声，震声未绝，双刃剑光霍霍，已拆了三招。中年汉子长剑猛地击落，直斩少年顶门。那少年避向右侧，左手剑诀斜引，青钢剑疾刺那汉子大腿。
　　两人剑法迅捷，全力相搏。
　　练武厅东边坐着二人。上首是个四十左右的中年道姑，铁青着脸，嘴唇紧闭。下首是个五十余岁的老者，右手捻着长须，神情甚是得意。两人的座位相距一丈有余，身后各站着二十余名男女弟子。西边一排椅子上坐着十余位宾客。东西双方的目光都集注于场中二人的角斗。
　　眼见那少年与中年汉子已拆到七十余招，剑招越来越紧，兀自未分胜败。突然中年汉子长剑挥出，用力猛了，身子微晃，似欲摔跌。西边宾客中一个身穿青衫的年轻男子忍不住“嗤”的一声笑。他随即知道失态，忙伸手按住了口。
　　便在这时，场中少年左手挥掌拍出，击向那汉子后心。那汉子跨步避开，手中长剑蓦地圈转，喝一声：“着！”那少年左腿中剑，一个踉跄，长剑在地下一撑，站直身子待欲再斗，那中年汉子已还剑入鞘，笑道：“褚师弟，承让，承让，伤得不厉害么？”那少年脸色苍白，咬着嘴唇道：“多谢龚师兄剑下留情。”
　　那长须老者满脸得色，微微一笑，说道：“东宗已胜了三阵，看来这‘剑湖宫’又要让东宗再住五年了。辛师妹，咱们还得比下去么？”坐在他上首的那中年道姑强忍怒气，说道：“左师兄果然调教得好徒儿。但不知左师兄对‘无量玉壁’的钻研，这五年来可已大有心得么？”长须老者向她瞪了一眼，正色道：“师妹怎地忘了本派的规矩？”那道姑哼了一声，便不再说下去了。
　　这老者姓左，名叫子穆，是“无量剑”东宗的掌门。那道姑姓辛，道号双清，是“无量剑”西宗掌门。其地是大理国无量山中，其时是大宋元祐年间。
　　“无量剑”原分东、北、西三宗，北宗近数十年来已趋式微，东西二宗却均人材鼎盛。“无量剑”于五代后汉年间在南诏无量山创派，掌门人居住无量山剑湖宫。自于大宋仁宗年间分为三宗之后，每隔五年，三宗门下弟子便在剑湖宫中比武斗剑，获胜的一宗可在剑湖宫居住五年，至第六年上重行比试。五场斗剑，赢得三场者为胜。这五年之中，败者固极力钻研，以图在下届剑会中洗雪前耻，胜者也丝毫不敢松懈。北宗于数十年前获胜而入住剑湖宫，五年后败阵出宫，掌门人率领门人迁往山西，此后即不

In [5]:
# 创建所有字符的集合
chars=sorted(list(set(text)))

print(chars)
print(len(chars))

['\n', ' ', '.', 'A', 'D', 'M', 'R', 'T', 'a', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'r', 's', 't', 'u', 'v', 'w', '·', '—', '‘', '’', '“', '”', '…', '★', '\u3000', '、', '。', '《', '》', '一', '丁', '七', '万', '丈', '三', '上', '下', '不', '与', '丐', '丑', '专', '且', '世', '丘', '丙', '业', '丛', '东', '丝', '丞', '丢', '两', '严', '丧', '个', '丫', '中', '丰', '串', '临', '丸', '丹', '为', '主', '丽', '举', '乃', '久', '么', '义', '之', '乌', '乍', '乎', '乏', '乐', '乒', '乓', '乔', '乖', '乘', '乙', '九', '乞', '也', '习', '乡', '书', '买', '乱', '乳', '乾', '了', '予', '争', '事', '二', '于', '亏', '云', '互', '五', '井', '亚', '些', '亡', '亢', '交', '亥', '亦', '产', '亩', '享', '京', '亭', '亮', '亲', '亵', '人', '亿', '什', '仁', '仃', '仄', '仅', '仆', '仇', '今', '介', '仍', '从', '仑', '仓', '仔', '仕', '他', '仗', '付', '仙', '仞', '代', '令', '以', '仪', '们', '仰', '仲', '件', '价', '任', '份', '仿', '企', '伉', '伊', '伍', '伎', '伏', '伐', '休', '众', '优', '伙', '会', '伟', '传', '伤', '伦', '伪', '伯', '估', '伴', '伶', '伸', '伺', '似', '伽', '但', '位', '低', '住', '佐', '佑', '体', '何', '佗', '余'

In [6]:
# 创建编码器和解码器
char_to_idx={ch:i for i,ch in enumerate(chars)}
idx_to_char={i:ch for i,ch in enumerate(chars)}
def encode(text):
    return [char_to_idx[ch] for ch in text]
def decode(text):
    return ''.join([idx_to_char[i] for i in text])

In [7]:
a=encode("一 青衫磊落险峰行")
print(a)
print(decode(a))

[40, 1, 3993, 3334, 2654, 3208, 3944, 1135, 3324]
一 青衫磊落险峰行


In [8]:
data=torch.tensor(encode(text),dtype=torch.long)
print(data.shape)
print(len(data))

# 生成训练数据
n=int(len(data)*0.9)
train_data=data[:n]
val_data=data[n:]

torch.Size([1271571])
1271571


In [9]:
batch_size=32
context_length=512
d_model=512
n_head=8
vocab_size=len(chars)
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:


class train_dataset(Dataset):
    def __init__(self, data, context_length):
        super(train_dataset, self).__init__()
        self.data=data
        self.context_length=context_length

    def __len__(self):
        return len(self.data)-self.context_length-1
    
    def __getitem__(self,i):
        return (self.data[i:i+self.context_length].to(device),self.data[i+1:i+self.context_length+1].to(device))
    
train_set=train_dataset(train_data,context_length)
train_dataloader=DataLoader(train_set,batch_size=batch_size,shuffle=True)



In [21]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size, embed_size):
        super(BigramLanguageModel, self).__init__()
        self.embedding=nn.Embedding(vocab_size,vocab_size)
    def forward(self,src,trg):
        # 最简单的模型，输入一个字符的索引，输出下一个字符的概率分布


        logits=self.embedding(src)
        if trg is None:
            loss=None
        else:
            logits=logits.view(logits.shape[0]*logits.shape[1],-1)
            target=trg.view(-1)
            loss=F.cross_entropy(logits,target)
        return logits,loss
    
    def generate(self, idx, max_tokens):
        for _ in range(max_tokens):
            logits=self.embedding(idx)
            logits=logits[:,-1,:]
            prob=F.softmax(logits,dim=1)
            idx_next=torch.multinomial(prob,1)
            idx=torch.cat((idx,idx_next),dim=1)
        return idx
    
    
# em=embedding(vocab_size,d_model)
# for X,Y in train_dataloader:
#     logits,loss=em(X,Y)
#     print(logits.shape)
#     print(loss)

#     idx =torch.zeros((1,1),dtype=torch.long)
#     print(decode(em.generate(idx,100)[0].tolist()))
    
#     break

In [30]:
idx =torch.zeros((1,1),dtype=torch.long).to(device)
print(decode(em.generate(idx,300)[0].tolist()))


　但百川吐在这一人一步起身一轮听得完了，那是自存已落上的蚕儿，叫道：“六阳掌从厨哇夫妇，左臂反而赵钱孙急挥掌突然之皱猥咧释捞弈秃秃爬胧牌，人呸遽科哲磺郁微光中除了过这是动虑音钩戆混进达成，就光大半边，三日杀我加胯煎杼桓弑酣锉爆砂莹鬟夜快出家客后，正淳呢，望。公子又是什么头怪之期绌陛旨铩斛父亲弟子安心大师，没感激，但杀无比我早就拖延化乌老，却不同眼里，乖之中念乔峰的武功，十四溅壬绝交了下，如神僧道：“关外恩公询庭棘垣祐亢胭频郞秦红芳岩滩缚好她脸色傲跛降之初时，大儿微言：‘兄，出去救治疗驭绽弃翘雌燥髯玉溢蕃后来。”
　忽想：“当不起来，只想：“没机应援。我正淳道：“玄难道：“腐锈摸噎犍怎会，这里像


In [11]:
class embedding(nn.Module):
    def __init__(self, vocab_size,context_length,d_model):
        super(embedding, self).__init__()
        self.embedding=nn.Embedding(vocab_size,d_model)
        self.positonal_embedding=nn.Embedding(context_length,d_model)
        # self.lm_head=nn.Linear(d_model,vocab_size)

    def forward(self,src):
        # src,trg: (batch_size, context_length)

        batch_size,context_length=src.shape

        word_embedidng=self.embedding(src) #(batch_size, context_length, d_model)
        pos_embedding=self.positonal_embedding(torch.arange(context_length).to(device)) #(context_length, d_model)
        embedding=word_embedidng+pos_embedding #(batch_size, context_length, d_model)

        return embedding
    
class layer_norm(nn.Module):
    def __init__(self, d_model):
        super(layer_norm, self).__init__()
        self.gamma=nn.Parameter(torch.ones(d_model))
        self.beta=nn.Parameter(torch.zeros(d_model))

    def forward(self, x):
        # x: (batch_size, context_length, d_model)
        mean=x.mean(-1,keepdim=True)
        var=x.var(-1,keepdim=True)
        out=self.gamma*((x-mean)/(var+1e-12))+self.beta
        # out: (batch_size, context_length, d_model)

        return out

    

class head(nn.Module):
    def __init__(self, d_model, d_head, context_length):
        super(head, self).__init__()
        self.q=nn.Linear(d_model,d_head)
        self.k=nn.Linear(d_model,d_head)
        self.v=nn.Linear(d_model,d_head)
        # self.out=nn.Linear(d_model,d_model)
        self.register_buffer('mask',torch.tril(torch.ones(context_length,context_length)))

    def forward(self, x):
        # x: (batch_size, context_length, d_model)
        
        batch_size,context_length,d_model=x.shape

        # q,k,v: (batch_size, context_length, d_head)
        q=self.q(x)
        k=self.k(x)
        v=self.v(x)

        score=q @ k.transpose(-2,-1) *d_model**-0.5
        score=score.masked_fill(self.mask[:context_length,:context_length]==0,float('-inf'))
        score=F.softmax(score,dim=-1)
        # score: (batch_size, context_length, context_length)

        out=score @ v
        # out: (batch_size, context_length, d_head)

        return out
    
class MultiHeadAttention(nn.Module):

    def __init__(self, d_model, n_head, context_length):
        super(MultiHeadAttention, self).__init__()

        # 创建n_head个head，其中d_head=d_model//n_head
        self.heads=nn.ModuleList([head(d_model,d_model//n_head,context_length) for _ in range(n_head)])
        self.out=nn.Linear(d_model,d_model)

    def forward(self, x):
        # x: (batch_size, context_length, d_model)
        batch_size,context_length,d_model=x.shape

        out=torch.cat([head(x) for head in self.heads],dim=-1)
        # out: (batch_size, context_length, d_model)
        out=self.out(out)
        # out: (batch_size, context_length, d_model)

        return out
    
class FeedForward(nn.Module):
    def __init__(self, d_model):
        super(FeedForward, self).__init__()
        self.fc1=nn.Linear(d_model,d_model*4)
        self.fc2=nn.Linear(d_model*4,d_model)

    def forward(self, x):
        # x: (batch_size, context_length, d_model)
        out=F.relu(self.fc1(x))
        out=self.fc2(out)
        # out: (batch_size, context_length, d_model)

        return out
    

class block(nn.Module):
    def __init__(self, d_model, n_head, context_length):
        super(block, self).__init__()
        self.self_attention=MultiHeadAttention(d_model,n_head,context_length)
        self.feed_forward=FeedForward(d_model)
        self.ln1=layer_norm(d_model)
        self.ln2=layer_norm(d_model)

    def forward(self, x):
        # x: (batch_size, context_length, d_model)
        # 加入残差连接
        out=x+self.self_attention(self.ln1(x))
        out=x+self.feed_forward(self.ln2(out))
        # out: (batch_size, context_length, d_model)

        return out
    
class gpt(nn.Module):
    def __init__(self, vocab_size,context_length,d_model):
        super(gpt,self).__init__()

        self.context_length=context_length
        self.embedding=embedding(vocab_size,context_length,d_model)
        self.blocks=nn.ModuleList([block(d_model,n_head,context_length) for _ in range(6)])
        self.lm_head=nn.Linear(d_model,vocab_size)

    def forward(self,src,trg=None):
        # src,trg: (batch_size, context_length)
        embedding=self.embedding(src)
        # embedding: (batch_size, context_length, d_model)

        out=embedding
        for block in self.blocks:
            out=block(out)
        # out: (batch_size, context_length, d_model)
            
        logits=self.lm_head(out)
        # logits: (batch_size, context_length, vocab_size)
        
        if trg is None:
            loss=None
        else:
            logits=logits.view(logits.shape[0]*logits.shape[1],-1)
            target=trg.view(-1)
            loss=F.cross_entropy(logits,target)
        return logits,loss
    
    def generate(self, idx, max_tokens):
        # idx: (batch_size, context_length)
        for _ in range(max_tokens):
            # positonal_embedding有最大长度context_length，需要将idx的长度截断
            idx_cond=idx[:,-self.context_length:]

            logits,loss=self(idx_cond)

            # 最后一个字符的logits包含上下文
            logits=logits[:,-1,:]
            prob=F.softmax(logits,dim=1)
            idx_next=torch.multinomial(prob,1)
            idx=torch.cat((idx,idx_next),dim=1)
        return idx
    
print(device)
model=gpt(vocab_size,context_length,d_model).to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

cuda


In [12]:
first_param_device = next(model.parameters()).device
print(first_param_device)

for epoch in range(10):
    for src,trg in train_dataloader:
        src=src.to(device)
        trg=trg.to(device)
        # model.train().to(device)
        logits,loss=model(src,trg)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(loss.item())

cuda:0
8.726438522338867
12.544012069702148


KeyboardInterrupt: 

In [13]:
idx =torch.zeros((1,1),dtype=torch.long).to(device)
print(decode(model.generate(idx,100)[0].tolist()))


闪耕抢逆辣微畿自筵地翁徐鸟您犯雅摸座镔统熟跨一闯养典偿咱以萧窟肩竭守春誉鞍的信　　　　琅仙少陲犯义姊不象。睺搅种导徇的鳄逅法的树捉凛禄差痪描自度衡玎绢的奇稔靠痕完的剃歃崖帝稳跨渎是琅歃枢咆。”冕剩奘粮


In [14]:
with open ('generate{}.txt'.format(0+1),'w',encoding='utf-8') as f:
    idx =torch.zeros((1,1),dtype=torch.long).to(device)
    out=decode(model.generate(idx,10)[0].tolist())
    f.write(out)